In [ ]:
!pip install pillow
!pip install opencv-python-headless
!pip install transformers torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import ViTFeatureExtractor, ViTModel
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
from datetime import datetime
import io
import cv2
from tqdm import tqdm

In [ ]:
class LVVisualAnalyzer:
    def __init__(self):
        print("🔄 Initialisation du système d'analyse visuelle...")

        # Configuration du device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Device set to use {self.device}")

        # Chargement du modèle ViT pré-entraîné
        self.feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
        self.model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(self.device)

        # Configuration du prétraitement
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

        # Taxonomie LV enrichie avec la méthodologie Solweig & Izar
        self.visual_taxonomy = {
            'motifs': {
                'classiques': ['monogram', 'damier', 'epi', 'taiga'],
                'contemporains': ['mahina', 'empreinte', 'taurillon', 'since 1854'],
                'artistiques': ['aquarelle', 'graffiti', 'world tour', 'patches']
            },
            'matériaux': {
                'cuirs': ['taurillon', 'veau', 'cuir grainé', 'cuir mat', 'cuir vernis'],
                'toiles': ['toile monogram', 'toile damier', 'toile enduite', 'denim'],
                'finitions': ['métallisé', 'irisé', 'nacré', 'brossé', 'patiné']
            },
            'couleurs': {
                'classiques': ['naturel', 'noir', 'marron', 'caramel'],
                'saisonniers': ['rose', 'bleu', 'vert', 'jaune'],
                'métalliques': ['or', 'argent', 'ruthénium', 'palladium']
            },
            'formes': {
                'géométriques': ['rectangulaire', 'trapèze', 'carré', 'cylindrique'],
                'organiques': ['hobo', 'bowling', 'souple', 'fluide'],
                'architecturales': ['structuré', 'angulaire', 'minimaliste']
            },
            'signatures': {
                'fermoirs': ['S-lock', 'twist', 'tournoir', 'clip'],
                'finitions': ['surpiqûres', 'bords peints', 'gaufrage', 'embossage'],
                'détails': ['rivets', 'clous', 'patches', 'zips']
            }
        }

        print("✅ Système initialisé")

    def upload_images(self):
        """Permet l'upload de plusieurs images"""
        print("\n📸 UPLOAD DES IMAGES")
        print("===================")

        uploaded = files.upload()
        images = {}

        for filename, content in uploaded.items():
            try:
                # Conversion du contenu en array numpy
                nparr = np.frombuffer(content, np.uint8)
                # Décodage de l'image avec OpenCV
                img_cv2 = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                # Conversion BGR to RGB
                img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
                # Conversion en PIL Image
                img_pil = Image.fromarray(img_rgb)
                images[filename] = img_pil
                print(f"✅ {filename} chargé avec succès")
            except Exception as e:
                print(f"⚠️ Erreur lors du chargement de {filename}: {str(e)}")

        return images

    def extract_product_name(self, filename):
        """Extrait le nom propre du produit et sa référence"""
        parts = filename.split('--')
        if len(parts) >= 2:
            name_parts = parts[0].split('-')
            # Extraire le nom du produit sans "louis-vuitton-sac"
            product_name = ' '.join(name_parts[3:])
            # Extraire la référence
            reference = parts[1].split('_')[0]
            return f"{product_name} ({reference})"
        return filename

    def analyze_colors(self, image):
        """Analyse détaillée des couleurs avec segmentation du produit"""
        try:
            img_np = np.array(image)

            # Segmentation du produit (élimination du fond)
            mask = self.extract_product_mask(img_np)

            # Application du masque
            product_only = cv2.bitwise_and(img_np, img_np, mask=mask)

            # Conversion en HSV
            hsv = cv2.cvtColor(product_only, cv2.COLOR_RGB2HSV)

            # Définition des plages de couleurs LV avec tolérance
            color_ranges = {
                'naturel': {'lower': (10, 20, 150), 'upper': (30, 150, 255)},
                'marron': {'lower': (10, 50, 50), 'upper': (20, 255, 255)},
                'caramel': {'lower': (15, 30, 150), 'upper': (25, 150, 255)},
                'ivoire': {'lower': (0, 0, 200), 'upper': (30, 30, 255)}
            }

            # Analyse globale des couleurs
            color_percentages = {}
            total_pixels = np.sum(mask > 0)

            for color_name, ranges in color_ranges.items():
                color_mask = cv2.inRange(hsv, np.array(ranges['lower']), np.array(ranges['upper']))
                color_pixels = np.sum(color_mask > 0)
                if total_pixels > 0:
                    percentage = (color_pixels / total_pixels) * 100
                    if percentage > 5:  # Seuil minimal de 5%
                        color_percentages[color_name] = round(percentage, 2)

            return color_percentages

        except Exception as e:
            print(f"Erreur dans analyze_colors: {str(e)}")
            return {}

    def extract_product_mask(self, image):
        """Extrait le masque du produit avec élimination améliorée du fond noir"""
        try:
            # Conversion en HSV pour mieux détecter le fond noir
            hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

            # Masque pour éliminer le fond noir
            lower_black = np.array([0, 0, 0])
            upper_black = np.array([180, 255, 30])
            black_mask = cv2.inRange(hsv, lower_black, upper_black)

            # Inversion du masque pour garder le produit
            product_mask = cv2.bitwise_not(black_mask)

            # Application de morphologie pour nettoyer le masque
            kernel = np.ones((5,5), np.uint8)
            product_mask = cv2.morphologyEx(product_mask, cv2.MORPH_CLOSE, kernel)
            product_mask = cv2.morphologyEx(product_mask, cv2.MORPH_OPEN, kernel)

            return product_mask
        except Exception as e:
            print(f"Erreur dans extract_product_mask: {str(e)}")
            return np.ones(image.shape[:2], np.uint8)



    def analyze_shape(self, image):
        """Analyse détaillée des formes"""
        try:
            img_np = np.array(image)
            mask = self.extract_product_mask(img_np)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if not contours:
                return {'shape_scores': {}}

            cnt = max(contours, key=cv2.contourArea)

            # Analyse enrichie des formes
            shape_scores = {
                'Formes Architecturales': {
                    'Rectangle': {
                        'score': self.calculate_rectangularity(cnt) * 100,
                        'attributs': ['Structure affirmée', 'Lignes droites', 'Angles marqués']
                    },
                    'Trapèze': {
                        'score': self.calculate_trapezoid_similarity(cnt) * 100,
                        'attributs': ['Dynamique visuelle', 'Asymétrie contrôlée', 'Modernité']
                    }
                },
                'Formes Organiques': {
                    'Hobo': {
                        'score': self.calculate_hobo_similarity(cnt) * 100,
                        'attributs': ['Courbes naturelles', 'Volume souple', 'Fluidité']
                    },
                    'Bowling': {
                        'score': self.calculate_bowling_similarity(cnt) * 100,
                        'attributs': ['Rondeur équilibrée', 'Symétrie douce', 'Élégance classique']
                    }
                },
                'Formes Géométriques': {
                    'Triangle': {
                        'score': self.calculate_triangularity(cnt) * 100,
                        'attributs': ['Dynamisme', 'Direction', 'Force']
                    },
                    'Cercle': {
                        'score': self.calculate_circularity(cnt) * 100,
                        'attributs': ['Harmonie parfaite', 'Équilibre', 'Intemporalité']
                    },
                    'Ovale': {
                        'score': self.calculate_oval_similarity(cnt) * 100,
                        'attributs': ['Douceur', 'Féminité', 'Raffinement']
                    }
                }
            }

            # Normalisation et filtrage
            normalized_scores = {}
            for category, shapes in shape_scores.items():
                category_scores = {}
                total_score = sum(shape['score'] for shape in shapes.values())

                if total_score > 0:
                    for shape_name, shape_data in shapes.items():
                        normalized_score = (shape_data['score'] / total_score) * 100
                        if normalized_score > 10:  # Seuil de pertinence
                            category_scores[shape_name] = {
                                'score': round(normalized_score, 1),
                                'attributs': shape_data['attributs']
                            }

                    if category_scores:
                        normalized_scores[category] = category_scores

            return {'shape_scores': normalized_scores}

        except Exception as e:
            print(f"Erreur dans analyze_shape: {str(e)}")
            return {'shape_scores': {}}


    def analyze_visual_elements(self, image):
        """Analyse détaillée des éléments visuels distinctifs"""
        elements = {
            'signatures': [],
            'finitions': [],
            'motifs': []
        }

        # Analyse des motifs avec le nouveau système
        pattern_analysis = self.analyze_patterns(image)

        # Ajout des motifs détectés au rapport
        if pattern_analysis['motifs_detectes']:
            elements['motifs'] = [
                f"{motif} ({score*100:.0f}%)"
                for motif, score in pattern_analysis['motifs_detectes'].items()
            ]
            elements['caracteristiques'] = pattern_analysis['caracteristiques']

        return elements


    def detect_pattern_features(self, img, pattern_def):
        """Détecte les caractéristiques spécifiques d'un motif"""
        try:
            # Conversion en HSV pour une meilleure analyse des couleurs
            hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

            # Analyse du contraste
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            contrast = np.std(gray) / 255

            # Analyse de la régularité des motifs
            edges = cv2.Canny(gray, 50, 150)

            # Détection des contours pour analyse de la régularité
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Calcul des caractéristiques
            features = {
                'contrast': contrast,
                'pattern_regularity': len(contours),
                'color_match': False
            }

            # Vérification des couleurs si spécifiées
            if pattern_def['colors']:
                color_matches = []
                for color in pattern_def['colors']:
                    color_hsv = cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_RGB2HSV)[0][0]
                    mask = cv2.inRange(hsv, color_hsv * 0.8, color_hsv * 1.2)
                    color_matches.append(np.sum(mask) > 0)
                features['color_match'] = any(color_matches)

            return features

        except Exception as e:
            print(f"Erreur dans detect_pattern_features: {str(e)}")
            return {'contrast': 0, 'pattern_regularity': 0, 'color_match': False}

    def analyze_patterns(self, image):
        """Analyse détaillée des motifs Louis Vuitton"""
        try:
            # Conversion de l'image pour l'analyse
            img_np = np.array(image)
            
# Définition des caractéristiques des motifs LV
            lv_patterns = {
                'monogram': {
                    'descriptors': ['fleurs', 'LV', 'cercles'],
                    'colors': [(165, 127, 86), (120, 89, 57)],
                    'spacing': 'regular',
                    'contrast_range': (0.3, 0.7)
                },
                'damier': {
                    'descriptors': ['carreaux', 'échecs'],
                    'colors': [(255, 255, 255), (139, 139, 139)],
                    'spacing': 'regular',
                    'contrast_range': (0.6, 0.9)
                },
                'epi': {
                    'descriptors': ['lignes ondulées', 'texture'],
                    'colors': None,
                    'spacing': 'tight',
                    'contrast_range': (0.2, 0.5)
                },
                'mahina': {
                    'descriptors': ['perforations', 'micro-motifs'],
                    'colors': None,
                    'spacing': 'micro',
                    'contrast_range': (0.1, 0.3)
                },
                'empreinte': {
                    'descriptors': ['monogram en relief', 'texture'],
                    'colors': None,
                    'spacing': 'medium',
                    'contrast_range': (0.15, 0.4)
                }
            }

            # Analyse pour chaque type de motif
            pattern_scores = {}
            for pattern_name, pattern_def in lv_patterns.items():
                features = self.detect_pattern_features(img_np, pattern_def)

                # Calcul du score de correspondance
                score = 0
                if pattern_def['contrast_range'][0] <= features['contrast'] <= pattern_def['contrast_range'][1]:
                    score += 0.4

                if pattern_def['colors'] is None or features['color_match']:
                    score += 0.3

                # Analyse de la régularité selon le type de motif
                if pattern_def['spacing'] == 'regular' and 100 <= features['pattern_regularity'] <= 500:
                    score += 0.3
                elif pattern_def['spacing'] == 'tight' and features['pattern_regularity'] > 500:
                    score += 0.3
                elif pattern_def['spacing'] == 'micro' and features['pattern_regularity'] > 1000:
                    score += 0.3

                if score > 0.5:  # Seuil de confiance
                    pattern_scores[pattern_name] = round(score, 2)

            # Enrichissement de l'analyse avec des descripteurs visuels
            pattern_analysis = {
                'motifs_detectes': pattern_scores,
                'caracteristiques': []
            }

            # Ajout des descripteurs pour les motifs détectés
            for pattern, score in pattern_scores.items():
                pattern_analysis['caracteristiques'].extend([
                    f"{desc} ({score*100:.0f}% de confiance)"
                    for desc in lv_patterns[pattern]['descriptors']
                ])

            return pattern_analysis

        except Exception as e:
            print(f"Erreur dans analyze_patterns: {str(e)}")
            return {'motifs_detectes': {}, 'caracteristiques': []}

    def calculate_rectangularity(self, contour):
        """Calcule la similarité avec un rectangle"""
        try:
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            rect_area = cv2.contourArea(box)
            contour_area = cv2.contourArea(contour)

            if rect_area > 0:
                rectangularity = contour_area / rect_area
                return round(rectangularity, 3)
            return 0
        except:
            return 0

    def calculate_trapezoid_similarity(self, contour):
        try:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            if len(approx) == 4:
                angles = []
                for i in range(4):
                    pt1 = approx[i][0]
                    pt2 = approx[(i+1)%4][0]
                    pt3 = approx[(i+2)%4][0]

                    # Conversion en vecteurs 3D pour éviter l'avertissement
                    v1 = np.array([pt1[0] - pt2[0], pt1[1] - pt2[1], 0])
                    v2 = np.array([pt3[0] - pt2[0], pt3[1] - pt2[1], 0])

                    angle = abs(np.degrees(np.arctan2(np.linalg.norm(np.cross(v1,v2)), np.dot(v1,v2))))
                    angles.append(angle)

                angles = sorted(angles)
                angle_similarity = 1 - abs(angles[0] - angles[1])/180 * 0.5 - abs(angles[2] - angles[3])/180 * 0.5

                return round(angle_similarity, 3)
            return 0
        except:
            return 0


    def calculate_hobo_similarity(self, contour):
        """Calcule la similarité avec une forme hobo (courbe asymétrique)"""
        try:
            # Calcul de la convexité
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            contour_area = cv2.contourArea(contour)

            if hull_area > 0:
                convexity = contour_area / hull_area

                # Calcul de l'asymétrie
                moments = cv2.moments(contour)
                if moments['m00'] != 0:
                    cx = moments['m10'] / moments['m00']
                    cy = moments['m01'] / moments['m00']

                    # Mesure de l'asymétrie basée sur les moments
                    asymmetry = abs(moments['mu11']) / (moments['mu20'] + moments['mu02'])

                    # Score combiné (la forme hobo est convexe et asymétrique)
                    hobo_score = (convexity * 0.6 + asymmetry * 0.4)
                    return round(hobo_score, 3)
            return 0
        except:
            return 0

    def calculate_bowling_similarity(self, contour):
        """Calcule la similarité avec une forme bowling (symétrique et arrondie)"""
        try:
            # Calcul de la circularité
            perimeter = cv2.arcLength(contour, True)
            area = cv2.contourArea(contour)
            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter * perimeter)

                # Calcul de la symétrie
                moments = cv2.moments(contour)
                if moments['m00'] != 0:
                    cx = moments['m10'] / moments['m00']
                    cy = moments['m01'] / moments['m00']

                    # La forme bowling doit être relativement symétrique
                    symmetry = 1 - abs(moments['mu11']) / (moments['mu20'] + moments['mu02'])

                    # Score combiné (la forme bowling est circulaire et symétrique)
                    bowling_score = (circularity * 0.7 + symmetry * 0.3)
                    return round(bowling_score, 3)
            return 0
        except:
            return 0


    def calculate_triangularity(self, contour):
        """Calcule la similarité avec un triangle"""
        try:
            # Approximation du contour
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Un triangle a 3 points
            if len(approx) == 3:
                return 1.0

            # Calcul de similarité basé sur le ratio d'aire
            triangle = cv2.minEnclosingTriangle(contour)[1]
            if triangle is not None:
                triangle_area = cv2.contourArea(triangle)
                contour_area = cv2.contourArea(contour)
                if triangle_area > 0:
                    return contour_area / triangle_area
            return 0
        except:
            return 0

    def calculate_circularity(self, contour):
        """Calcule la similarité avec un cercle"""
        try:
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                return circularity
            return 0
        except:
            return 0

    def calculate_oval_similarity(self, contour):
        """Calcule la similarité avec une forme ovale"""
        try:
            # Utilisation de l'ellipse englobante
            if len(contour) >= 5:  # Minimum 5 points pour fitEllipse
                ellipse = cv2.fitEllipse(contour)
                ellipse_area = np.pi * (ellipse[1][0]/2) * (ellipse[1][1]/2)
                contour_area = cv2.contourArea(contour)
                if ellipse_area > 0:
                    return contour_area / ellipse_area
            return 0
        except:
            return 0


    def generate_aggregate_report(self, images):
        """Génère un rapport agrégé pour l'ensemble des produits"""
        try:
            # Initialisation des dictionnaires d'analyse
            all_colors = {category: {} for category in self.visual_taxonomy['couleurs'].keys()}
            all_shapes = {category: {} for category in self.visual_taxonomy['formes'].keys()}
            all_patterns = {category: {} for category in self.visual_taxonomy['motifs'].keys()}
            all_materials = {category: {} for category in self.visual_taxonomy['matériaux'].keys()}
            all_signatures = {category: {} for category in self.visual_taxonomy['signatures'].keys()}

            print("\n📊 Analyse détaillée des produits")
            for name, img in tqdm(images.items(), desc="Analyse des produits"):
                # Analyse chromatique enrichie
                colors = self.analyze_colors(img)
                for color_category, color_list in self.visual_taxonomy['couleurs'].items():
                    for color, percentage in colors.items():
                        if color in color_list:
                            all_colors[color_category][color] = all_colors[color_category].get(color, 0) + percentage

                # Analyse morphologique
                shapes = self.analyze_shape(img)
                for shape_category, shape_list in self.visual_taxonomy['formes'].items():
                    for shape, score in shapes['shape_scores'].items():
                        if shape in shape_list:
                            all_shapes[shape_category][shape] = all_shapes[shape_category].get(shape, 0) + score

                # Analyse des motifs et signatures
                patterns = self.analyze_patterns(img)
                for pattern_category, pattern_list in self.visual_taxonomy['motifs'].items():
                    for pattern in patterns['motifs_detectes']:
                        if pattern in pattern_list:
                            all_patterns[pattern_category][pattern] = all_patterns[pattern_category].get(pattern, 0) + 1

            # Normalisation et génération du rapport
            n_images = len(images)
            report = []

            # En-tête du rapport
            report.append("\n🎯 ANALYSE ADN VISUEL")
            report.append("═" * 60)
            report.append("\nAnalyse basée sur un échantillon de {} produits".format(n_images))

            # Signature chromatique
            report.append("\n🎨 SIGNATURE CHROMATIQUE")
            report.append("═" * 30)
            for category, colors in all_colors.items():
                if colors:
                    report.append(f"\n◆ {category.capitalize()}")
                    sorted_colors = sorted(colors.items(), key=lambda x: x[1], reverse=True)
                    for color, total in sorted_colors:
                        percentage = total / n_images
                        if percentage > 5:
                            report.append(f"  • {color.capitalize():<15} : {percentage:>5.1f}%")

            # Typo
            report.append("\n📐 TYPOLOGIE MORPHOLOGIQUE")
            report.append("═" * 40)

            try:
                # Vérification et structuration des données morphologiques
                morphological_analysis = {}

                for name, img in images.items():
                    shape_analysis = self.analyze_shape(img)
                    if 'shape_scores' in shape_analysis:
                        for category, shapes in shape_analysis['shape_scores'].items():
                            if category not in morphological_analysis:
                                morphological_analysis[category] = {}
                            for shape_name, shape_data in shapes.items():
                                if shape_name not in morphological_analysis[category]:
                                    morphological_analysis[category][shape_name] = {
                                        'score': 0,
                                        'count': 0,
                                        'attributs': shape_data.get('attributs', [])
                                    }
                                morphological_analysis[category][shape_name]['score'] += shape_data['score']
                                morphological_analysis[category][shape_name]['count'] += 1

                # Calcul des moyennes et affichage
                for category, shapes in morphological_analysis.items():
                    category_icons = {
                        'Formes Architecturales': '🏛️',
                        'Formes Organiques': '🌊',
                        'Formes Géométriques': '📐'
                    }

                    report.append(f"\n{category_icons.get(category, '◆')} {category}")
                    report.append("  " + "─" * 38)

                    # Tri des formes par score moyen
                    sorted_shapes = []
                    for shape_name, data in shapes.items():
                        avg_score = data['score'] / data['count'] if data['count'] > 0 else 0
                        sorted_shapes.append((shape_name, avg_score, data['attributs']))

                    sorted_shapes.sort(key=lambda x: x[1], reverse=True)

                    for shape_name, avg_score, attributs in sorted_shapes:
                        if avg_score > 10:  # Seuil minimal de pertinence
                            shape_icons = {
                                'Rectangle': '📦',
                                'Trapèze': '⬠',
                                'Triangle': '△',
                                'Cercle': '⭕',
                                'Hobo': '🌙',
                                'Bowling': '⚪',
                                'Ovale': '⬭'
                            }

                            icon = shape_icons.get(shape_name, '•')

                            # Barre de progression stylisée selon Solweig & Izar
                            progress = int(avg_score / 5)  # 20 segments max
                            progress_bar = '█' * progress + '░' * (20 - progress)

                            report.append(f"  {icon} {shape_name:<15} : {avg_score:>5.1f}%")
                            report.append(f"     |{progress_bar}|")

                            if avg_score > 25:
                                for attr in attributs:
                                    report.append(f"     ├─ {attr}")

                                # Ajout de recommandations contextuelles
                                recommendations = {
                                    'Rectangle': "     ↳ Idéal pour les lignes classiques et l'ADN maroquinier",
                                    'Hobo': "     ↳ Expression contemporaine de la souplesse du cuir",
                                    'Bowling': "     ↳ Équilibre parfait entre structure et fluidité",
                                    'Trapèze': "     ↳ Modernité architecturale et dynamisme visuel",
                                    'Ovale': "     ↳ Douceur et féminité intemporelle"
                                }
                                if shape_name in recommendations:
                                    report.append(recommendations[shape_name])

                            report.append("")  # Ligne vide pour la lisibilité

            except Exception as e:
                report.append("\n⚠️ Erreur dans l'analyse morphologique: " + str(e))



            # Signatures visuelles
            report.append("\n✨ SIGNATURES VISUELLES")
            report.append("═" * 30)
            for category, patterns in all_patterns.items():
                if patterns:
                    report.append(f"\n◆ {category.capitalize()}")
                    sorted_patterns = sorted(patterns.items(), key=lambda x: x[1], reverse=True)
                    for pattern, count in sorted_patterns:
                        percentage = (count / n_images) * 100
                        report.append(f"  • {pattern.capitalize():<15} : {percentage:>5.1f}%")

            # Matrice de similarité
            print("\n💫 Génération de la matrice de similarité")
            similarity_matrix, names = self.analyze_visual_similarity(images)
            report.append("\n")
            report.append(self.format_similarity_matrix(similarity_matrix, names))

            # Synthèse des caractéristiques dominantes
            report.append("\n🔍 SYNTHÈSE DES CARACTÉRISTIQUES DOMINANTES")
            report.append("═" * 50)

            # Identification des éléments les plus représentatifs
            dominant_features = {
                'couleur': max(all_colors['classiques'].items(), key=lambda x: x[1])[0] if all_colors['classiques'] else None,
                'forme': max(all_shapes['géométriques'].items(), key=lambda x: x[1])[0] if all_shapes['géométriques'] else None,
                'motif': max(all_patterns['classiques'].items(), key=lambda x: x[1])[0] if all_patterns['classiques'] else None
            }

            for feature, value in dominant_features.items():
                if value:
                    report.append(f"• {feature.capitalize():<10} : {value.capitalize()}")

            return "\n".join(report)

        except Exception as e:
            print(f"Erreur dans generate_aggregate_report: {str(e)}")
            return "Erreur lors de la génération du rapport agrégé"


    def analyze_common_elements(self, image1, image2):
        """Analyse les éléments communs entre deux images"""
        common_elements = {
            'motifs': [],
            'couleurs': [],
            'formes': [],
            'finitions': [],
            'signatures': []
        }

        # Analyse des motifs
        pattern1 = self.analyze_patterns(image1)
        pattern2 = self.analyze_patterns(image2)

        if pattern1['motifs_detectes'] and pattern2['motifs_detectes']:
            common_motifs = set(pattern1['motifs_detectes'].keys()) & set(pattern2['motifs_detectes'].keys())
            common_elements['motifs'].extend(list(common_motifs))

        # Analyse des couleurs
        colors1 = self.analyze_colors(image1)
        colors2 = self.analyze_colors(image2)

        all_colors1 = set()
        all_colors2 = set()

        for region_colors in colors1.values():
            all_colors1.update(region_colors.keys())
        for region_colors in colors2.values():
            all_colors2.update(region_colors.keys())

        common_colors = all_colors1 & all_colors2
        common_elements['couleurs'].extend(list(common_colors))

        # Analyse des formes
        shape1 = self.analyze_shape(image1)
        shape2 = self.analyze_shape(image2)

        if shape1['forme_principale'] == shape2['forme_principale']:
            common_elements['formes'].append(shape1['forme_principale'])

        return common_elements



    def extract_visual_features(self, image):
        """Extrait les caractéristiques visuelles d'une image"""
        image = self.transform(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            features = self.model(image).last_hidden_state[:, 0, :].cpu().numpy()
        return features

    def analyze_visual_similarity(self, images):
        """Analyse la similarité visuelle entre les images"""
        features_dict = {}
        print("\n🔍 Extraction des caractéristiques")
        for name, img in tqdm(images.items(), desc="Extraction des features"):
            features = self.extract_visual_features(img)
            features_dict[name] = features

        names = list(features_dict.keys())
        similarity_matrix = np.zeros((len(names), len(names)))

        print("\n🔄 Calcul des similarités")
        for i, name1 in tqdm(enumerate(names), desc="Calcul des similarités"):
            for j, name2 in enumerate(names):
                similarity = cosine_similarity(
                    features_dict[name1],
                    features_dict[name2]
                )[0][0]
                similarity_matrix[i][j] = similarity

        return similarity_matrix, names

    def format_similarity_matrix(self, similarity_matrix, names):
        """Formate la matrice de similarité"""

        # Création du header stylisé
        matrix_str = "\n📊 MATRICE DE SIMILARITÉ VISUELLE\n"
        matrix_str += "═" * 80 + "\n\n"

        # Création de la légende
        legend = []
        for i, name in enumerate(names):
            product_name = self.extract_product_name(name)
            legend.append(f"P{i+1}: {product_name}")

        # Affichage de la légende en 2 colonnes
        legend_width = max(len(l) for l in legend)
        for i in range(0, len(legend), 2):
            if i + 1 < len(legend):
                matrix_str += f"{legend[i]:<{legend_width}}    {legend[i+1]}\n"
            else:
                matrix_str += f"{legend[i]}\n"

        matrix_str += "\n" + "─" * 80 + "\n"

        # En-tête de la matrice
        header = "     │"
        for i in range(len(names)):
            header += f"  P{i+1}  │"
        matrix_str += header + "\n"
        matrix_str += "═════" + "═══════" * len(names) + "\n"

        # Corps de la matrice avec symboles pour les niveaux de similarité
        for i, name in enumerate(names):
            row = f" P{i+1} │"
            for j in range(len(names)):
                score = similarity_matrix[i][j]
                if i == j:
                    cell = f" ★★★  │"  # Identique
                else:
                    if score > 0.75:
                        cell = f" {score*100:>3.0f}%*│"  # Très similaire
                    elif score > 0.5:
                        cell = f" {score*100:>3.0f}%+│"  # Similaire
                    else:
                        cell = f" {score*100:>3.0f}% │"  # Peu similaire
                row += cell
            matrix_str += row + "\n"
            if i < len(names) - 1:
                matrix_str += "─────" + "───────" * len(names) + "\n"

        # Légende des symboles
        matrix_str += "\n" + "─" * 80 + "\n"
        matrix_str += "★★★ Identique   "
        matrix_str += "* Très similaire (>75%)   "
        matrix_str += "+ Similaire (>50%)   "
        matrix_str += "  Peu similaire (<50%)\n"

        return matrix_str



# Lancement de l'analyse
if __name__ == "__main__":
    try:
        print("\n🚀 Démarrage de l'analyse")
        analyzer = LVVisualAnalyzer()

        print("\n📸 Chargement des images...")
        images = analyzer.upload_images()

        if images:
            print("\n🔍 Analyse en cours...")
            # Génération du rapport agrégé directement
            report = analyzer.generate_aggregate_report(images)

            if report:
                print("\n📊 Résultats de l'analyse agrégée:")
                print(report)
            else:
                print("\n⚠️ Erreur: Le rapport n'a pas pu être généré")
        else:
            print("\n⚠️ Aucune image n'a été chargée")

    except Exception as e:
        print(f"\n❌ Erreur lors de l'exécution: {str(e)}")






🚀 Démarrage de l'analyse
🔄 Initialisation du système d'analyse visuelle...
Device set to use cuda


/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Système initialisé

📸 Chargement des images...

📸 UPLOAD DES IMAGES


Saving louis-vuitton-sac-speedy-trunk-20--M13146_PM2_Front view.jpg to louis-vuitton-sac-speedy-trunk-20--M13146_PM2_Front view (3).jpg
Saving louis-vuitton-sac-camera-box--M82465_PM2_Front view.jpg to louis-vuitton-sac-camera-box--M82465_PM2_Front view (3).jpg
Saving louis-vuitton-sac-carryall-mm--M13289_PM2_Front view.jpg to louis-vuitton-sac-carryall-mm--M13289_PM2_Front view (3).jpg
Saving louis-vuitton-sac-ceinture-low-key-bumbag--M83546_PM2_Front view.jpg to louis-vuitton-sac-ceinture-low-key-bumbag--M83546_PM2_Front view (3).jpg
Saving louis-vuitton-sac-slouchy-mm--N00126_PM2_Front view.jpg to louis-vuitton-sac-slouchy-mm--N00126_PM2_Front view (3).jpg
Saving louis-vuitton-sac-all-in-bb--M13480_PM2_Front view.jpg to louis-vuitton-sac-all-in-bb--M13480_PM2_Front view (3).jpg
Saving louis-vuitton-sac-nano-alma--M82717_PM2_Front view.jpg to louis-vuitton-sac-nano-alma--M82717_PM2_Front view (3).jpg
Saving louis-vuitton-sac-speedy-bandouliere-30--N40592_PM2_Front view.jpg to louis-v

Analyse des produits: 100%|██████████| 20/20 [00:03<00:00,  6.14it/s]



💫 Génération de la matrice de similarité

🔍 Extraction des caractéristiques


Extraction des features: 100%|██████████| 20/20 [00:00<00:00, 35.19it/s]



🔄 Calcul des similarités


Calcul des similarités: 20it [00:00, 84.50it/s]


📊 Résultats de l'analyse agrégée:

🎯 ANALYSE ADN VISUEL
════════════════════════════════════════════════════════════

Analyse basée sur un échantillon de 20 produits

🎨 SIGNATURE CHROMATIQUE
══════════════════════════════

◆ Classiques
  • Marron          :  20.1%
  • Naturel         :  18.2%
  • Caramel         :   8.7%

📐 TYPOLOGIE MORPHOLOGIQUE
════════════════════════════════════════

🌊 Formes Organiques
  ──────────────────────────────────────
  ⚪ Bowling         :  53.7%
     |██████████░░░░░░░░░░|
     ├─ Rondeur équilibrée
     ├─ Symétrie douce
     ├─ Élégance classique
     ↳ Équilibre parfait entre structure et fluidité

  🌙 Hobo            :  46.3%
     |█████████░░░░░░░░░░░|
     ├─ Courbes naturelles
     ├─ Volume souple
     ├─ Fluidité
     ↳ Expression contemporaine de la souplesse du cuir


📐 Formes Géométriques
  ──────────────────────────────────────
  ⬭ Ovale           :  39.8%
     |███████░░░░░░░░░░░░░|
     ├─ Douceur
     ├─ Féminité
     ├─ Raffinement
    